In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [17]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
prod_type = "PE"
# prod_type = "CE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


In [18]:
#########################
#User-defined functions #
#########################

def quad_func(x, a0, a1, a2):
    y = a0 + a1*x + a2*(np.square(x))
    return y

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       batch_size_divisor=10, no_of_epochs = 1000, stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    quad_polyn_volSurf = pd.read_csv(output_path + "A3_" + stock_ident + "_quadPolyn_vol_surface_params.csv")
    
    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week] 
            K = strike          
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]


            df_polyn_date = quad_polyn_volSurf[quad_polyn_volSurf['Date'] == sim_start_date.strftime("%d-%b-%Y")]

            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
            price_moneyness = option_moneyness
    
            t_list = np.sort(np.unique(np.array(df_polyn_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
            
            df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
            polyn_coeff = df_polyn_coeff[["a0", "a1", "a2"]].to_numpy().reshape(-1)

            x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)
            

            if (sim_moneyness <= x_min):
                vol_list = [quad_func(x_min, *polyn_coeff)]
            elif (sim_moneyness >= x_max):
                vol_list = [quad_func(x_max, *polyn_coeff)]
            else:
                vol_list = [quad_func(sim_moneyness, *polyn_coeff)]

            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            put_strikes = np.sort(np.array(dict_pe_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
                        
            stock_vec = sim_stock_mat[:, 1]
            r_f = get_risk_free_rate_from_exact_date(sim_start_date)
            dt = float((each_month - sim_end_date).days) / 365

            # Need higher tenor in the first element to avoiding tenor_list[1] as missing when 2 tenors are not available
            t_list = -np.sort(np.unique(np.array(-df_polyn_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]            

            df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
            polyn_coeff = df_polyn_coeff[["a0", "a1", "a2"]].to_numpy().reshape(-1)
            
            x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)
            
            if (price_moneyness <= x_min):
                vol_list = [quad_func(x_min, *polyn_coeff)]
            elif (price_moneyness >= x_max):
                vol_list = [quad_func(x_max, *polyn_coeff)]
            else:
                vol_list = [quad_func(price_moneyness, *polyn_coeff)]
            
            vol = vol_list[0]
                       
            opt_strike = strike

            if (option_type == "CE" and week != no_of_weeks-1):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt)
            elif (option_type == "PE" and week != no_of_weeks-1):
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt) 
            elif (option_type == "CE" and week == no_of_weeks-1):
                option_value = np.maximum(stock_vec - opt_strike, 0) 
            else:
                option_value = np.maximum(opt_strike - stock_vec, 0)
    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 
            
            
#             print("SKLearn Linear regression - OLS")
#             lin_model = LinearRegression().fit(x, y)
#             pfl_weights = lin_model.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model.intercept_
#             print("R-Square: ", lin_model.score(x,y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)           
            
#             print("SKLearn Linear regression - Ridge")
#             lin_model_ridge = Ridge(alpha=1.0).fit(x, y)
#             pfl_weights = lin_model_ridge.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model_ridge.intercept_
#             print("R-Square: ", lin_model_ridge.score(x, y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)
            
            
            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)
            
#          
#             #Find Optimal Alpha
#             alphas = np.array([-0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()
            
#             print("Stats Model _ OLS")
#             x = x
#             x = sm.add_constant(x)
#             lin_model2 = OLS(y,x).fit()
#             print(lin_model2.summary())
# #             print(lin_model2.params)
# #           pfl_weights = np.array(lin_model2.params).reshape(-1)
            
            
            ######################################################################################
            ### Portfolio weights of shorter term options from neural network of European options ###
            ######################################################################################
            
            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
            
            
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

        df_ce_pfl_weights.to_csv(output_path  + "B16A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
        df_pe_pfl_weights.to_csv(output_path + "B16A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
        df_cash_pfl_weights.to_csv(output_path  + "B16A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
    df_hedged_prod.to_csv(output_path + "B16A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [19]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

# #Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

# # #Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 

# # # The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)


2019-08-29  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.996540282600691
Coefficients:
[-0.         -0.00043291 -0.003736   -0.         -0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.02148018  0.01084953  0.00370043  0.00530338  0.00306546
  0.00407173  0.00318616  0.00144426  0.00950382]
[1.86545132]
2019-08-29  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9991444931447416
Coefficients:
[-0.00960323 -0.         -0.          0.          0.          0.0054322
  0.          0.          0.          0.          0.          0.
  0.          0.          0.03402394  0.04323082  0.02016794  0.00951362
  0.00912969  0.00875631  0.00755604  0.0046605   0.00953033  0.01444568]
[3.54861122]
2019-08-29  Week:  2
['6D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9993235269953368
Coefficients:
[-0.00226211 -0.00225298 -0.002368

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 227.43939708171098, tolerance: 170.12470619047838
  positive)


R-Square:  0.9994276598045331
Coefficients:
[-0.00679177 -0.00136787 -0.          0.          0.          0.00115456
  0.00309985  0.00214538  0.00018882  0.          0.          0.
  0.          0.          0.04264341  0.10841085  0.07034488  0.05654908
  0.04340942  0.0246536   0.01267822  0.00964828  0.0074064   0.01182576
  0.00829286  0.0103814 ]
[2.94875434]
2019-10-31  Week:  3
['7D', '14D']
SKLearn Linear regression - Lasso
R-Square:  0.9807708894711398
Coefficients:
[-0.         -0.00012575 -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.01454693  0.00676618  0.00379454  0.00313561  0.00203738
  0.0017438   0.00146265  0.00035886  0.          0.00198844]
[0.07118551]
2019-10-31  Week:  4
['7D']
SKLearn Linear regression - Lasso
R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


2019-11-28  Week:  0
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9991496319516612
Coefficients:
[-0.0030112  -0.00230163 -0.00226984 -0.         -0.          0.
  0.          0.00222002  0.00237591  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.04709632  0.
  0.01402734  0.0096233   0.00853877  0.0081245   0.00566985  0.00547481
  0.0094165   0.00351914  0.00327708  0.01075511]
[3.4600919]
2019-11-28  Week:  1
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9942340073213339
Coefficients:
[-0.         -0.         -0.         -0.00086575 -0.         -0.
 -0.         -0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.01238516  0.00768355  0.00419411  0.00404367  0.00325812
  0.00225966  0.00209341  0.00206155  0.00078307  0.          0.00425844]
[0.40561733]
2019-11-28  Week:  2
['7

...

R-Square:  0.9986405910311164
Coefficients:
[-0.         -0.00011565 -0.         -0.         -0.         -0.
 -0.         -0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.06561549
  0.09664165  0.03663102  0.0367621   0.0253059   0.0185949   0.01228936
  0.00850814  0.0040881   0.00153015  0.00187477  0.00147726  0.00084855
  0.00027601  0.00012034  0.00055735  0.00161354]
[0.08347771]
2020-06-25  Week:  3
['7D']
SKLearn Linear regression - Lasso
R-Square:  0.9958110627507729
Coefficients:
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 378.88337905605704, tolerance: 266.3565328266571
  positive)


R-Square:  0.9992138651819829
Coefficients:
[-0.00287602 -0.00130957 -0.00028001 -0.         -0.          0.
  0.          0.          0.00074836  0.00175095  0.00059915  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.00489339  0.04622461  0.01292924  0.00890746  0.00784036
  0.00818055  0.00727013  0.00543588  0.00446043  0.0050687   0.00441996
  0.00277355  0.00330339  0.00481485  0.00300351  0.00745064]
[2.46243588]
2020-07-30  Week:  3
['7D', '14D']
SKLearn Linear regression - Lasso


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.743150124909988, tolerance: 3.5496056569961287
  positive)
/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


R-Square:  0.993014631527623
Coefficients:
[-0.00000000e+00 -0.00000000e+00 -1.16726408e-04 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.39241115e-03  5.07471474e-02
  8.58690239e-03  8.55782980e-03  6.07573373e-03  5.08774651e-03
  4.09422341e-03  3.27891454e-03  2.70931950e-03  1.93395254e-03
  1.44501972e-03  1.26845681e-03  1.15936020e-03  6.26448916e-04
  0.00000000e+00  8.69518869e-05  1.70631579e-03]
[0.07822839]
2020-07-30  Week:  4
['7D']
SKLearn Linear regression - Lasso
R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0.]
